In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/월간데이터7/data.zip'

Archive:  /content/drive/MyDrive/월간데이터7/data.zip
  inflating: submission.csv          
  inflating: test.csv                
  inflating: train.csv               


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
import cv2

import gc
from keras import backend as bek

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')

In [5]:
x_train = train.drop(['id', 'digit', 'letter'], 1).to_numpy()
x_train = x_train.reshape(-1,28,28,1)
x_train = x_train/255

y = train['digit']
y_train = np.zeros((len(y), len(y.unique()))) # (2048,10)


#keras의 to_categorical 함수를 사용해서 바로 one-hot encoding 가능하다.

for i, digit in enumerate(y):
    y_train[i, digit] = 1 #one-hot-encoding을 해주는 건가



데이터 증가시키기 ==>  image augmentation

keras 

model.complie() 

: http://blog.naver.com/PostView.nhn?blogId=shwotjd14&logNo=221458224334&categoryNo=0&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView

models.Model()

: https://frhyme.github.io/machine-learning/a_model_in_keras/

: 케라스에서 모델 설계를 할 때 보통은 Sequential()을 사용하는데, 단순히 이전 layer의 output이 다음 layer의 input으로 들어가는 선형적인 경우에 편리 -> densenet()같이 선형적 흐름이 아닌 모델 설계는 사용 못 함
multi input, multi output 지원함

*지금 코드에서는 선형적,, Sequential 사용해도 상관 없는 듯



In [6]:
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

def create_model(x_train):

    inputs = tf.keras.Input(x_train.shape[1:])

    bn = tf.keras.layers.BatchNormalization()(inputs)
    conv = tf.keras.layers.Conv2D(32, kernel_size=5, strides=1, padding='same', activation='relu')(bn)
    bn = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(64, kernel_size=5, strides=1, padding='same', activation='relu')(bn)
    pool = tf.keras.layers.MaxPooling2D((2, 2))(conv)

    bn = tf.keras.layers.BatchNormalization()(pool)
    conv = tf.keras.layers.Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    bn = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(256, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    pool = tf.keras.layers.MaxPooling2D((2, 2))(conv)

    flatten = tf.keras.layers.Flatten()(pool)

    den = tf.keras.layers.Dense(1024, activation='relu')(flatten)
    drop = tf.keras.layers.Dropout(0.2)(den)
    out = tf.keras.layers.Dense(10, activation='softmax')(drop)

    model = tf.keras.models.Model(inputs = inputs, outputs=out) # input이랑 output parameter로 넣어주면 모델 인스턴스화

    return model







In [7]:
model = create_model(x_train)
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=50)

Epoch 1/50
64/64 [==============================] - 8s 8ms/step - loss: 3.0669 - accuracy: 0.1745
Epoch 2/50
64/64 [==============================] - 0s 7ms/step - loss: 1.5233 - accuracy: 0.4738
Epoch 3/50
64/64 [==============================] - 0s 7ms/step - loss: 1.1564 - accuracy: 0.5955
Epoch 4/50
64/64 [==============================] - 0s 7ms/step - loss: 0.9685 - accuracy: 0.6631
Epoch 5/50
64/64 [==============================] - 0s 7ms/step - loss: 0.7098 - accuracy: 0.7632
Epoch 6/50
64/64 [==============================] - 0s 7ms/step - loss: 0.6533 - accuracy: 0.7690
Epoch 7/50
64/64 [==============================] - 0s 7ms/step - loss: 0.4856 - accuracy: 0.8440
Epoch 8/50
64/64 [==============================] - 0s 7ms/step - loss: 0.3973 - accuracy: 0.8663
Epoch 9/50
64/64 [==============================] - 0s 7ms/step - loss: 0.2651 - accuracy: 0.9232
Epoch 10/50
64/64 [==============================] - 0s 7ms/step - loss: 0.1909 - accuracy: 0.9511
Epoch 11/50
64/64 [

In [8]:
x_test = test.drop(['id', 'letter'],1).to_numpy()
x_test = x_test.reshape(-1,28,28,1)
x_test = x_test/255


submission['digit'] = np.argmax(model.predict(x_test), axis=1)
submission.head()

,id,digit
0,2049,6
1,2050,9
2,2051,1
3,2052,0
4,2053,3


In [9]:
submission.to_csv('cv_ver3.csv',index=False)
#0.81372  모델 conv2d()이거만 바꿨는데 이케 됐따?